### Project title
# Development of a Bimodal Biometric Crime Control System using a Modified Neura Network

Bimodal biometric systems represent a strategic evolution in crime control technologies. By leveraging the complementary strengths of face and fingerprint recognition, and harnessing the power of deep learning, these systems offer a robust, reliable, and efficient solution for identity verification in high-stakes security environments.

# Training Strategy
### Dataset Preparation
Biometric Databases: Use comprehensive datasets that include both modalities, ensuring they represent real-world variability.

### Data Augmentation
Augment the training set to simulate variations such as different angles, lighting conditions, and partial occlusions. Techniques may include rotation, scaling, and translation.
Labeling and Ground Truth: Ensure that each sample is correctly labeled to supervise training.

### Training Process
Loss Functions: Use cross-entropy loss for classification tasks, potentially combined with additional regularization terms to prevent overfitting.
Optimization: Experiment with optimizers like Adam or SGD with momentum. Fine-tune learning rates and implement learning rate decay strategies.
Validation: Use a dedicated validation set to monitor model performance and prevent overfitting.

### Evaluation Metrics and Testing
Accuracy: The overall rate of correct identifications.
Receiver Operating Characteristic (ROC) Curve: Used to evaluate the trade-off between true positive rate and false positive rate.
Confusion Matrix: Provides detailed insights into classification errors.

In [6]:
# Import required libraries
import os
import cv2
import numpy as np
from glob import glob

# Dataset Preparation

Load and preprocess BMP images from the given folder.
Images are read in grayscale, resized, and normalized.

In [21]:
def load_images_from_folder(folder, resize_shape=(128, 128)):
    images = {}
    # Get all BMP files (case-insensitive)
    file_paths = glob(os.path.join(folder, "*.BMP"))
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue  # Skip if image cannot be read
        img = cv2.resize(img, resize_shape)
        img = img / 255.0  # Normalize pixel values to [0, 1]
        images[file_name] = img
    return images

### Organize and match face and fingerprint images.

Iterates through each subject folder in base_path (e.g., '1' to '45'),
loads fingerprint, left face, and right face images, and creates samples.
Each sample consists of one fingerprint image paired with a representative
left and right face image for that subject.

In [24]:
def structure_dataset(base_path):
    dataset = []
    
    # Get a sorted list of all subject folders in the base path
    subject_folders = sorted([d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))])
    
    for subject in subject_folders:
        subject_path = os.path.join(base_path, subject)
        fingerprint_folder = os.path.join(subject_path, "Fingerprint")
        left_folder = os.path.join(subject_path, "left")
        right_folder = os.path.join(subject_path, "right")
        
        # Load images from each modality
        fingerprints = load_images_from_folder(fingerprint_folder)
        left_faces = load_images_from_folder(left_folder)
        right_faces = load_images_from_folder(right_folder)
        
        # Skip subject if any modality is missing
        if not fingerprints or not left_faces or not right_faces:
            print(f"Skipping subject {subject}: Missing modality")
            continue
        
        # Choose a representative left and right face image (first sorted image)
        left_face_img = left_faces[sorted(left_faces.keys())[0]]
        right_face_img = right_faces[sorted(right_faces.keys())[0]]
        
        # For each fingerprint image, create a sample with the corresponding face images
        for fp_name, fp_img in fingerprints.items():
            sample = {
                "id": subject,
                "fingerprint": fp_img,
                "left_face": left_face_img,
                "right_face": right_face_img,
                "fp_name": fp_name
            }
            dataset.append(sample)
    
    return dataset

# Set the base path to the folder containing the dataset
base_path = "Project Assignments/Tumininu Akibowale/IRIS and FINGERPRINT DATASET"

dataset = structure_dataset(base_path)
print("Total samples processed:", len(dataset))
dataset[:3]

Total samples processed: 450


[{'id': '1',
  'fingerprint': array([[0.62745098, 0.62745098, 0.62745098, ..., 0.        , 0.        ,
          0.        ],
         [0.62745098, 0.53333333, 0.4745098 , ..., 0.        , 0.        ,
          0.        ],
         [0.62745098, 0.49411765, 0.5254902 , ..., 0.        , 0.        ,
          0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
          0.        ]]),
  'left_face': array([[0.96862745, 0.9372549 , 0.95294118, ..., 0.59607843, 0.58039216,
          0.55294118],
         [0.91372549, 0.95686275, 0.9254902 , ..., 0.60392157, 0.58823529,
          0.55294118],
         [0.93333333, 0.97254902, 0.97254902, ..., 0.59607843, 0.59607843,
          0.57647059],
         ...,
         [0.83529412, 0.85490196, 0.89803922, .

# Data Splitting and Augmentation

We create a BimodalBiometricDataset class that converts the preprocessed NumPy arrays (for fingerprint, left face, and right face) into PIL images, applies transforms (including data augmentation for training), and assigns a numerical label to each subject.

In [26]:
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms

class BimodalBiometricDataset(Dataset):
    def __init__(self, data, transform=None):

        self.data = data
        self.transform = transform
        # Create a mapping from subject id to a numeric label.
        self.subjects = sorted(list(set(item["id"] for item in data)))
        self.subject2label = {subject: idx for idx, subject in enumerate(self.subjects)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        # Each modality is stored as a NumPy array in the range [0, 1]
        fingerprint = sample["fingerprint"]
        left_face = sample["left_face"]
        right_face = sample["right_face"]

        # Convert numpy arrays to PIL images (we assumes images are grayscale)
        fingerprint_img = Image.fromarray((fingerprint * 255).astype(np.uint8), mode='L')
        left_face_img = Image.fromarray((left_face * 255).astype(np.uint8), mode='L')
        right_face_img = Image.fromarray((right_face * 255).astype(np.uint8), mode='L')

        # Apply the transform (data augmentation)
        if self.transform:
            fingerprint_img = self.transform(fingerprint_img)
            left_face_img = self.transform(left_face_img)
            right_face_img = self.transform(right_face_img)
        else:
            # Default conversion to tensor if no transform is provided
            fingerprint_img = transforms.ToTensor()(fingerprint_img)
            left_face_img = transforms.ToTensor()(left_face_img)
            right_face_img = transforms.ToTensor()(right_face_img)

        # Get the numeric label for the subject
        label = self.subject2label[sample["id"]]
        return {"fingerprint": fingerprint_img, 
                "left_face": left_face_img, 
                "right_face": right_face_img, 
                "label": label}

# Define transforms
# For training: include data augmentation (random horizontal flip, rotation) and conversion to tensor.
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

# Split the Data: 
Divide your dataset into training, validation, and testing sets (common splits are 70% for training, 15% for validation, and 15% for testing). This ensures you can both train your model and evaluate its performance on unseen data.

In [28]:
# Create the dataset object using the train transforms initially.
dataset_obj = BimodalBiometricDataset(dataset, transform=train_transforms)

# Data splitting: 70% train, 15% validation, 15% test.
total_samples = len(dataset_obj)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)
test_size = total_samples - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset_obj, [train_size, val_size, test_size])

print(f"Total samples: {total_samples}")
print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")


Total samples: 450
Train samples: 315, Validation samples: 67, Test samples: 68


# Data Augmentation: 
Apply augmentation techniques (e.g., rotations, translations, scaling) to the training data. This is particularly important because our dataset is limited, as it will help improve the model's robustness and prevent overfitting.

In [30]:
# For validation/test: simply convert to tensor.
test_transforms = transforms.ToTensor()

# For validation and test sets, override the transform to remove augmentation.
# Create new dataset instances for validation and test with test_transforms.
# (Since random_split returns Subset objects, we can manually set their dataset.transform.)
train_dataset.dataset.transform = train_transforms  # training with augmentation
val_dataset.dataset.transform = test_transforms      # no augmentation for validation
test_dataset.dataset.transform = test_transforms     # no augmentation for testing

# Create DataLoaders
batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Test one batch to verify the outputs
for batch in train_loader:
    print("Fingerprint batch shape:", batch['fingerprint'].shape)
    print("Left face batch shape:", batch['left_face'].shape)
    print("Right face batch shape:", batch['right_face'].shape)
    print("Labels:", batch['label'])
    break


Fingerprint batch shape: torch.Size([8, 1, 128, 128])
Left face batch shape: torch.Size([8, 1, 128, 128])
Right face batch shape: torch.Size([8, 1, 128, 128])
Labels: tensor([19, 33,  5,  8, 13, 12, 40,  4])


# Data Modelling

We will create neural network that processes three modalities (left face, right face, and fingerprint), fuses their features, and produces a classification output. Each branch uses a simple convolutional neural network (CNN) that extracts a 128-dimensional feature vector. These three feature vectors are then concatenated into a 384-dimensional vector, which is passed through fully connected layers to generate the final predictions.

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# CNN for processing face images (used for both left and right views)
class FaceCNN(nn.Module):
    def __init__(self):
        super(FaceCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        # Input shape: (batch, 1, 128, 128)
        x = self.pool(F.relu(self.conv1(x)))  # -> (batch, 32, 64, 64)
        x = self.pool(F.relu(self.conv2(x)))  # -> (batch, 64, 32, 32)
        x = self.pool(F.relu(self.conv3(x)))  # -> (batch, 128, 16, 16)
        # Use adaptive average pooling to reduce spatial dimensions to 1x1
        x = F.adaptive_avg_pool2d(x, (1, 1))  # -> (batch, 128, 1, 1)
        x = x.view(x.size(0), -1)             # -> (batch, 128)
        return x

# CNN for processing fingerprint images (similar structure)
class FingerprintCNN(nn.Module):
    def __init__(self):
        super(FingerprintCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        # Input shape: (batch, 1, 128, 128)
        x = self.pool(F.relu(self.conv1(x)))  # -> (batch, 32, 64, 64)
        x = self.pool(F.relu(self.conv2(x)))  # -> (batch, 64, 32, 32)
        x = self.pool(F.relu(self.conv3(x)))  # -> (batch, 128, 16, 16)
        x = F.adaptive_avg_pool2d(x, (1, 1))    # -> (batch, 128, 1, 1)
        x = x.view(x.size(0), -1)               # -> (batch, 128)
        return x

# Combined network that fuses features from both face branches and the fingerprint branch.
class BiometricNet(nn.Module):
    def __init__(self, num_classes):
        super(BiometricNet, self).__init__()
        # Instantiate separate CNNs for left and right face images.
        self.left_face_cnn = FaceCNN()
        self.right_face_cnn = FaceCNN()
        # Instantiate a CNN for fingerprint images.
        self.fingerprint_cnn = FingerprintCNN()
        # The feature dimension from each branch is 128. Combined, we have 128 * 3 = 384.
        self.fc = nn.Sequential(
            nn.Linear(384, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, left_face, right_face, fingerprint):
        # Process each modality
        left_feat = self.left_face_cnn(left_face)       # (batch, 128)
        right_feat = self.right_face_cnn(right_face)      # (batch, 128)
        fp_feat = self.fingerprint_cnn(fingerprint)       # (batch, 128)
        # Concatenate the features from all three branches
        combined = torch.cat((left_feat, right_feat, fp_feat), dim=1)  # (batch, 384)
        out = self.fc(combined)  # (batch, num_classes)
        return out


In [34]:

import torch.optim as optim
from tqdm import tqdm

# Device configuration: use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Number of classes (subjects)
num_classes = 45

# Instantiate the model and move it to the device
model = BiometricNet(num_classes=num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training settings
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_train = 0
    correct_train = 0
    
    # Training loop
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        # Get data and move to device
        left_face = batch['left_face'].to(device)
        right_face = batch['right_face'].to(device)
        fingerprint = batch['fingerprint'].to(device)
        labels = batch['label'].to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(left_face, right_face, fingerprint)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate loss and correct predictions
        running_loss += loss.item() * labels.size(0)
        total_train += labels.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += torch.sum(preds == labels).item()
    
    epoch_loss = running_loss / total_train
    epoch_acc = (correct_train / total_train) * 100
    print(f"Epoch {epoch+1}: Train Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.2f}%")
    
    # Validation loop
    model.eval()
    running_val_loss = 0.0
    total_val = 0
    correct_val = 0
    
    with torch.no_grad():
        for batch in val_loader:
            left_face = batch['left_face'].to(device)
            right_face = batch['right_face'].to(device)
            fingerprint = batch['fingerprint'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(left_face, right_face, fingerprint)
            loss = criterion(outputs, labels)
            
            running_val_loss += loss.item() * labels.size(0)
            total_val += labels.size(0)
            _, preds = torch.max(outputs, 1)
            correct_val += torch.sum(preds == labels).item()
    
    val_loss = running_val_loss / total_val
    val_acc = (correct_val / total_val) * 100
    print(f"Epoch {epoch+1}: Val Loss = {val_loss:.4f}, Accuracy = {val_acc:.2f}%\n")


Using device: cuda


Epoch 1/20: 100%|██████████████████████████████████████████████████████████████████████| 40/40 [00:05<00:00,  7.86it/s]


Epoch 1: Train Loss = 3.8200, Accuracy = 0.95%
Epoch 1: Val Loss = 3.8146, Accuracy = 2.99%



Epoch 2/20: 100%|██████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  9.41it/s]


Epoch 2: Train Loss = 3.8038, Accuracy = 2.22%
Epoch 2: Val Loss = 3.8315, Accuracy = 0.00%



...

Epoch 18: Train Loss = 3.0684, Accuracy = 15.56%
Epoch 18: Val Loss = 3.2381, Accuracy = 20.90%



Epoch 19/20: 100%|█████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  9.34it/s]


Epoch 19: Train Loss = 2.8333, Accuracy = 20.32%
Epoch 19: Val Loss = 2.9942, Accuracy = 19.40%



Epoch 20/20: 100%|█████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  9.33it/s]


Epoch 20: Train Loss = 2.6692, Accuracy = 21.27%
Epoch 20: Val Loss = 2.9438, Accuracy = 16.42%



# Evaluate the Model

In [36]:
# Evaluate the model on the test set after training
model.eval()
running_test_loss = 0.0
total_test = 0
correct_test = 0

with torch.no_grad():
    for batch in test_loader:
        left_face = batch['left_face'].to(device)
        right_face = batch['right_face'].to(device)
        fingerprint = batch['fingerprint'].to(device)
        labels = batch['label'].to(device)
        
        outputs = model(left_face, right_face, fingerprint)
        loss = criterion(outputs, labels)
        
        running_test_loss += loss.item() * labels.size(0)
        total_test += labels.size(0)
        _, preds = torch.max(outputs, 1)
        correct_test += torch.sum(preds == labels).item()

test_loss = running_test_loss / total_test
test_acc = (correct_test / total_test) * 100
print(f"Test Loss = {test_loss:.4f}, Test Accuracy = {test_acc:.2f}%")


Test Loss = 2.8926, Test Accuracy = 16.18%


## Perform data uptimization

In [38]:

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Define model
num_classes = 45
model = BiometricNet(num_classes=num_classes).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)  # Slightly lower learning rate
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  # Reduce LR every 10 epochs

# Training settings
num_epochs = 30  # Increase to allow better learning

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        left_face = batch['left_face'].to(device)
        right_face = batch['right_face'].to(device)
        fingerprint = batch['fingerprint'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = model(left_face, right_face, fingerprint)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Track accuracy
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
        running_loss += loss.item() * labels.size(0)

    epoch_loss = running_loss / total_train
    epoch_acc = 100 * correct_train / total_train
    print(f"Epoch {epoch+1}: Train Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.2f}%")
    
    # Validation
    model.eval()
    val_loss, correct_val, total_val = 0.0, 0, 0
    
    with torch.no_grad():
        for batch in val_loader:
            left_face = batch['left_face'].to(device)
            right_face = batch['right_face'].to(device)
            fingerprint = batch['fingerprint'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(left_face, right_face, fingerprint)
            loss = criterion(outputs, labels)
            
            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)
            val_loss += loss.item() * labels.size(0)
    
    val_loss /= total_val
    val_acc = 100 * correct_val / total_val
    print(f"Epoch {epoch+1}: Val Loss = {val_loss:.4f}, Accuracy = {val_acc:.2f}%\n")

    scheduler.step()  # Adjust learning rate



Using device: cuda


Epoch 1/30: 100%|██████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  9.39it/s]


Epoch 1: Train Loss = 3.8137, Accuracy = 3.17%
Epoch 1: Val Loss = 3.8226, Accuracy = 0.00%



Epoch 2/30: 100%|██████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  9.39it/s]


Epoch 2: Train Loss = 3.8019, Accuracy = 2.22%
Epoch 2: Val Loss = 3.8446, Accuracy = 0.00%



...

Epoch 28: Train Loss = 2.4618, Accuracy = 28.57%
Epoch 28: Val Loss = 2.7861, Accuracy = 17.91%



Epoch 29/30: 100%|█████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  9.35it/s]


Epoch 29: Train Loss = 2.4010, Accuracy = 29.52%
Epoch 29: Val Loss = 2.6577, Accuracy = 22.39%



Epoch 30/30: 100%|█████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  9.39it/s]


Epoch 30: Train Loss = 2.3761, Accuracy = 28.89%
Epoch 30: Val Loss = 2.6505, Accuracy = 28.36%



In [28]:
# Final Test Evaluation
model.eval()
test_loss, correct_test, total_test = 0.0, 0, 0

with torch.no_grad():
    for batch in test_loader:
        left_face = batch['left_face'].to(device)
        right_face = batch['right_face'].to(device)
        fingerprint = batch['fingerprint'].to(device)
        labels = batch['label'].to(device)

        outputs = model(left_face, right_face, fingerprint)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        correct_test += (preds == labels).sum().item()
        total_test += labels.size(0)
        test_loss += loss.item() * labels.size(0)

test_loss /= total_test
test_acc = 100 * correct_test / total_test
print(f"Final Test Loss = {test_loss:.4f}, Test Accuracy = {test_acc:.2f}%")


Final Test Loss = 3.4716, Test Accuracy = 10.29%


# Applying Transfer Learning for Bimodal Biometrics
Since the model is not learning well using raw CNN architecture, we will use pre-trained 
networks to extract face and fingerprint features, and then train a new classifier on top.

Steps for Transfer Learning
Use a Pretrained Model for Face Recognition and Fingerprint Recognition

    ResNet-18 for Face Recognition

    MobileNetV2 for Fingerprint Recognition

    Extract fingerprint embeddings.

    Combine Features and Train a Final Classifier

    Feed extracted embeddings into a fully connected neural network (MLP).

    Train only the last layers, while freezing the backbone models.

## Load Image Paths and Labels
Since each subject has multiple images, we will pair each face image with all fingerprints.

In [39]:
import os

# Path to the dataset folder (update this if needed)
dataset_root = "Project Assignments\\Tumininu Akibowale\\IRIS and FINGERPRINT DATASET"

def get_image_paths_and_labels():
    image_pairs = []
    labels = []

    # Loop through each person's folder (1, 2, ..., 45)
    for person_id in sorted(os.listdir(dataset_root)):
        person_folder = os.path.join(dataset_root, person_id)

        if not os.path.isdir(person_folder):
            continue  # Skip if not a folder

        fingerprint_folder = os.path.join(person_folder, "Fingerprint")
        left_face_folder = os.path.join(person_folder, "left")
        right_face_folder = os.path.join(person_folder, "right")

        # Get all fingerprints
        fingerprint_images = [
            os.path.join(fingerprint_folder, f)
            for f in os.listdir(fingerprint_folder) if f.endswith(".BMP")
        ]

        # Get all left face images
        left_face_images = [
            os.path.join(left_face_folder, f)
            for f in os.listdir(left_face_folder) if f.endswith(".bmp")
        ]

        # Get all right face images
        right_face_images = [
            os.path.join(right_face_folder, f)
            for f in os.listdir(right_face_folder) if f.endswith(".bmp")
        ]

        # Pair each left face image with all fingerprints
        for face in left_face_images:
            for fingerprint in fingerprint_images:
                image_pairs.append((face, fingerprint))
                labels.append(int(person_id))

        # Pair each right face image with all fingerprints
        for face in right_face_images:
            for fingerprint in fingerprint_images:
                image_pairs.append((face, fingerprint))
                labels.append(int(person_id))

    return image_pairs, labels

# Load data
all_image_pairs, all_labels = get_image_paths_and_labels()

# Check dataset size
print(f"Total Samples: {len(all_image_pairs)}")
print(f"Unique Classes (Subjects): {len(set(all_labels))}")


Total Samples: 4500
Unique Classes (Subjects): 45


## Split Data for Training, Validation, and Testing
Since we have multiple samples per subject, we will split at the subject level

In [40]:
from sklearn.model_selection import train_test_split

# Get unique subjects
unique_labels = list(set(all_labels))

# Split subjects into train (70%), validation (15%), and test (15%)
train_subjects, test_subjects = train_test_split(unique_labels, test_size=0.3, random_state=42)
val_subjects, test_subjects = train_test_split(test_subjects, test_size=0.5, random_state=42)

# Function to filter pairs based on subjects
def filter_by_subject(image_pairs, labels, subjects):
    return [(img1, img2, label) for (img1, img2), label in zip(image_pairs, labels) if label in subjects]

train_data = filter_by_subject(all_image_pairs, all_labels, train_subjects)
val_data = filter_by_subject(all_image_pairs, all_labels, val_subjects)
test_data = filter_by_subject(all_image_pairs, all_labels, test_subjects)

print(f"Train Samples: {len(train_data)}, Val Samples: {len(val_data)}, Test Samples: {len(test_data)}")


Train Samples: 3100, Val Samples: 700, Test Samples: 700


In [50]:
# Extract all labels from dataset
num_classes = 45
train_labels = [label for _, _, label in train_data]
val_labels = [label for _, _, label in val_data]
test_labels = [label for _, _, label in test_data]

print("Unique Labels in Training Set:", set(train_labels))
print("Expected Class Range: 0 to", num_classes - 1)


Unique Labels in Training Set: {1, 2, 3, 6, 8, 10, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 28, 29, 30, 31, 32, 34, 35, 37, 38, 39, 41, 43, 45}
Expected Class Range: 0 to 44


In [52]:
train_labels = [label - 1 for label in train_labels]
val_labels = [label - 1 for label in val_labels]
test_labels = [label - 1 for label in test_labels]

print("Unique Labels in Training Set:", set(train_labels))

Unique Labels in Training Set: {0, 1, 2, 5, 7, 9, 10, 11, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 27, 28, 29, 30, 31, 33, 34, 36, 37, 38, 40, 42, 44}


## Create PyTorch Dataset Class
We will process both face and fingerprint images and use ResNet-18 (for faces) and MobileNetV2 (for fingerprints) as feature extractors.

In [54]:
import torchvision.models as models
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F

# Load pre-trained models
face_model = models.resnet18(pretrained=True)
# Modify MobileNetV2 to include global average pooling
fingerprint_model = models.mobilenet_v2(pretrained=True)
fingerprint_model.classifier = nn.Identity()  # Remove final classification layer

# Remove final classification layers (we only need feature extractors)
face_model = nn.Sequential(*list(face_model.children())[:-1])
fingerprint_model = nn.Sequential(*list(fingerprint_model.children())[:-1])

# Set models to evaluation mode (no training)
face_model.eval()
fingerprint_model.eval()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def extract_embedding(image_path, model):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        embedding = model(image)

    # Apply Global Average Pooling if necessary
    if len(embedding.shape) > 2:  # If it's not a 1D tensor
        embedding = torch.nn.functional.adaptive_avg_pool2d(embedding, (1, 1))

    return embedding.view(-1)  # Flatten to 1D


## Define PyTorch Dataset

In [56]:
sample_face, sample_fingerprint, _ = train_data[0]

face_embedding = extract_embedding(sample_face, face_model)
fingerprint_embedding = extract_embedding(sample_fingerprint, fingerprint_model)

print("Face Embedding Size:", face_embedding.shape)  # Should be [512]
print("Fingerprint Embedding Size:", fingerprint_embedding.shape)  # Should be [1280]

# Concatenate embeddings
features = torch.cat((face_embedding, fingerprint_embedding), dim=0)
print("Final Feature Vector Size:", features.shape)  # Should be [1792]


Face Embedding Size: torch.Size([512])
Fingerprint Embedding Size: torch.Size([1280])
Final Feature Vector Size: torch.Size([1792])


## Modify Dataset Class
Instead of feeding raw images into CNN, we now extract embeddings from the pretrained models.

In [83]:
class BimodalBiometricDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.image_paths = image_paths  # List of (face_path, fingerprint_path)
        self.labels = labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        face_path, fingerprint_path = self.image_paths[idx]
        label = self.labels[idx]

        # Extract embeddings
        face_embedding = extract_embedding(face_path, face_model)
        fingerprint_embedding = extract_embedding(fingerprint_path, fingerprint_model)

        # Concatenate both embeddings
        features = torch.cat((face_embedding, fingerprint_embedding), dim=0)

        return {'features': features, 'label': torch.tensor(label, dtype=torch.long)}


## Define a New Classifier
Since the feature extractors are frozen, we now train an MLP classifier on top.

In [71]:
class BiometricClassifier(nn.Module):
    def __init__(self, feature_size, num_classes):
        super(BiometricClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(feature_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.fc(x)


## Train the Classifier
Since embeddings are already extracted, we train only the classifier.

In [79]:
# Create dataset and dataloaders
train_dataset = BimodalBiometricDataset(train_data, train_labels)
val_dataset = BimodalBiometricDataset(val_data, val_labels)
test_dataset = BimodalBiometricDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Define MLP Classifier
class BiometricClassifier(nn.Module):
    def __init__(self, feature_size, num_classes):
        super(BiometricClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(feature_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.fc(x)

# Set up model
feature_size = 512 + 1280  # ResNet-18 output (512) + MobileNetV2 output (1280)
num_classes = 45
classifier = BiometricClassifier(feature_size, num_classes).to("cuda")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    classifier.train()
    running_loss, correct, total = 0.0, 0, 0
    
    for batch in train_loader:
        features = batch['features'].to("cuda")
        labels = batch['label'].to("cuda")

        optimizer.zero_grad()
        outputs = classifier(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Accuracy tracking
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        running_loss += loss.item() * labels.size(0)

    train_loss = running_loss / total
    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Accuracy = {train_acc:.2f}%")


ValueError: too many values to unpack (expected 2)

In [ ]:
# Final Evaluation
classifier.eval()
test_loss, correct_test, total_test = 0.0, 0, 0

with torch.no_grad():
    for batch in test_loader:
        features = batch['features'].to(device)
        labels = batch['label'].to(device)

        outputs = classifier(features)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        correct_test += (preds == labels).sum().item()
        total_test += labels.size(0)
        test_loss += loss.item() * labels.size(0)

test_loss /= total_test
test_acc = 100 * correct_test / total_test
print(f"Final Test Loss = {test_loss:.4f}, Test Accuracy = {test_acc:.2f}%")
